In [1]:
# Importar librería para la conexión con MySQL
# -----------------------------------------------------------------------
import mysql.connector
from mysql.connector import errorcode


# Importar librerías para manipulación y análisis de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

from tqdm import tqdm

In [ ]:
#Creo la BBDD:

cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1')


mycursor = cnx.cursor()
try:
    mycursor.execute("CREATE DATABASE ABC_corporation")
    print(mycursor)
except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

----
**Employee**

In [3]:

df = pd.read_csv('')


In [6]:
# Creo la tabla para las employees

cnx = mysql.connector.connect(
    user='root', password='AlumnaAdalab',
    host='127.0.0.1', database='ABC_corporation'
)

mycursor = cnx.cursor()


crear_tabla = """
CREATE TABLE IF NOT EXISTS employees (
    AGE int DEFAULT NULL,
    GENDER VARCHAR(50) DEFAULT NULL,
    MARITAL_STATUS VARCHAR(50) DEFAULT NULL,
    EDUCATION INT DEFAULT NULL,
    EDUCATION_FIELD VARCHAR(100) DEFAULT NULL,
    EMPLOYEE_NUMBER INT NOT NULL PRIMARY KEY
    )
    """

# Ejecuta la instrucción SQL para crear la tabla
try:
    mycursor.execute(crear_tabla)
    print("Tabla creada correctamente.")
except mysql.connector.Error as err:
    print("Error al crear la tabla:", err)

# Cierra el cursor y la conexión a la base de datos
mycursor.close()
cnx.close()

Tabla creada correctamente.


In [7]:
# Insertar información de las employees

cnx = mysql.connector.connect(
    user='root', password='AlumnaAdalab',
    host='127.0.0.1', database='ABC_corporation'
)

mycursor = cnx.cursor()

# Lee el archivo CSV
df = pd.read_csv("")

# Define el rango de filas sobre el cual iterar
inicio =   
fin = 

# Itera sobre el rango de filas especificado del DataFrame
for index in tqdm(range(inicio, fin), desc="Insertando registros"):
    row = df.iloc[index]  # Obtén la fila en el índice actual
    sql = "INSERT INTO employees (AGE, GENDER, MARITAL_STATUS, EDUCATION, EDUCATION_FIELD, EMPLOYEE_NUMBER) VALUES (%s, %s, %s, %s, %s, %s)"

    val = (row['AGE'], row['GENDER'], row['MARITAL_STATUS'], row['EDUCATION'], row['EDUCATION_FIELD'], row['EMPLOYEE_NUMBER'])

    try:
        mycursor.execute(sql, val)
        cnx.commit()  # Confirma la transacción
    except mysql.connector.Error as err:
        cnx.rollback()  # Revierte la transacción si hay un error
        print("Error al insertar registro:", err)

# Cierra el cursor y la conexión a la base de datos
mycursor.close()
cnx.close()

Insertando registros:  13%|█▎        | 66578/524321 [01:44<11:20, 672.79it/s]

Error al insertar registro: 1406 (22001): Data too long for column 'Titulo' at row 1


Insertando registros: 100%|█████████▉| 524320/524321 [13:19<00:00, 655.49it/s]


IndexError: single positional indexer is out-of-bounds

----
**Employees_job**

In [9]:
df = pd.read_csv('')

In [11]:
# Creo la tabla para employees_job

cnx = mysql.connector.connect(
    user='root', password='AlumnaAdalab',
    host='127.0.0.1', database='ABC_corporation'
)

mycursor = cnx.cursor()


crear_tabla = """
CREATE TABLE IF NOT EXISTS employees_job (
    EMPLOYEE_NUMBER INT NOT NULL,
    NUM_COMPANIES_WORKED INT DEFAULT NULL,
    RELATIONSHIP_SATISFACTION INT DEFAULT NULL,
    STOCK_OPTION_LEVEL INT DEFAULT NULL,
    TOTAL_WORKING_YEARS INT DEFAULT NULL,
    TRAINING_TIMES_LAST_YEAR INT DEFAULT NULL,
    WORK_LIFE_BALANCE INT DEFAULT NULL,
    YEARS_AT_COMPANY INT DEFAULT NULL,
    YEARS_SINCE_LAST_PROMOTION INT DEFAULT NULL,
    YEARS_WITH_CURRENT_MANAGER INT DEFAULT NULL,
    REMOTE_WORK INT(1) DEFAULT NULL,
    PRIMARY KEY (`EMPLOYEE_NUMBER`),
    FOREIGN KEY (EMPLOYEE_NUMBER) REFERENCES employees (EMPLOYEE_NUMBER) ON DELETE CASCADE ON UPDATE CASCADE
)
"""
# Puntuacion_IMDB, Direccion, Guionistas, Argumento, Duracion, Nombre_de_la_pelicula

try:
    mycursor.execute(crear_tabla)
    print("Tabla creada correctamente.")
except mysql.connector.Error as err:
    print("Error al crear la tabla:", err)


mycursor.close()
cnx.close()

Tabla creada correctamente.


In [16]:
#Insertar información employees_job

# Lee el archivo CSV
df = pd.read_csv("")

# Define el rango de filas sobre el cual iterar
inicio =   
fin = 

# Itera sobre el rango de filas especificado del DataFrame
for index in tqdm(range(inicio, fin), desc="Insertando registros"):
    row = df.iloc[index]  # Obtén la fila en el índice actual
    sql = "INSERT INTO employees (EMPLOYEE_NUMBER, NUM_COMPANIES_WORKED, RELATIONSHIP_SATISFACTION, STOCK_OPTION_LEVEL,TOTAL_WORKING_YEARS, TRAINING_TIMES_LAST_YEAR, WORK_LIFE_BALANCE, YEARS_AT_COMPANY, YEARS_SINCE_LAST_PROMOTION, YEARS_WITH_CURRENT_MANAGER, REMOTE_WORK) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"

   

    val = (row['EMPLOYEE_NUMBER'],row['NUM_COMPANIES_WORKED'] ,row['RELATIONSHIP_SATISFACTION'], row['STOCK_OPTION_LEVEL'], row['TOTAL_WORKING_YEARS'], row['TRAINING_TIMES_LAST_YEAR'], row['WORK_LIFE_BALANCE'], row['YEARS_AT_COMPANY'],  row['YEARS_SINCE_LAST_PROMOTION'], row['YEARS_WITH_CURRENT_MANAGER'], row['REMOTE_WORK'])

    try:
        mycursor.execute(sql, val)
        cnx.commit()  # Confirma la transacción
    except mysql.connector.Error as err:
        cnx.rollback()  # Revierte la transacción si hay un error
        print("Error al insertar registro:", err)

# Cierra el cursor y la conexión a la base de datos
mycursor.close()
cnx.close()

Insertando registros:  13%|█▎        | 66536/524320 [02:18<17:13, 442.83it/s]

Error al insertar registro: 1406 (22001): Data too long for column 'Titulo' at row 1


Insertando registros: 100%|██████████| 524320/524320 [15:19<00:00, 570.39it/s]


-----
**ACTORES**

In [18]:
#quitar duplicados de actores

df = pd.read_csv('union_actores_unicos.csv')

# Eliminar duplicados
new_df_no_duplicate = df.drop_duplicates()

# Guardar el DataFrame modificado en un nuevo archivo CSV
new_df_no_duplicate.to_csv('actores_sin_duplicados.csv', index=False)

In [17]:
# Creo la tabla actores

cnx = mysql.connector.connect(
    user='root', password='AlumnaAdalab',
    host='127.0.0.1', database='proyecto_cine'
)

mycursor = cnx.cursor()


crear_tabla = """
CREATE TABLE IF NOT EXISTS info_actores (
    Nombre VARCHAR(255),
    Nacimiento VARCHAR(255),
    Conocido VARCHAR(255),
    Que_hace TEXT,
    Premios VARCHAR(255),
    Oscar VARCHAR(255)
)
"""
# Nombre,Nacimiento,Conocido,Que hace,Premios,Oscar

try:
    mycursor.execute(crear_tabla)
    print("Tabla creada correctamente.")
except mysql.connector.Error as err:
    print("Error al crear la tabla:", err)


mycursor.close()
cnx.close()

Tabla creada correctamente.


In [20]:
#Insertar información IMDB

cnx = mysql.connector.connect(
    user='root', password='AlumnaAdalab',
    host='127.0.0.1', database='proyecto_cine'
)

mycursor = cnx.cursor()


df = pd.read_csv("actores_sin_duplicados.csv")
df.drop(df.columns[0], axis=1, inplace=True)

inicio = 0
fin = 36981


for index in tqdm(range(inicio, fin)):
   
    row = df.iloc[index]
    
    sql = "INSERT INTO info_actores (Nombre, Nacimiento, Conocido, Que_hace, Premios, Oscar) VALUES (%s, %s, %s, %s, %s, %s)"
    
    try:
        mycursor.execute(sql, (str(row['Nombre']), str(row['Nacimiento']), str(row['Conocido']), str(row['Que hace']), str(row['Premios']), str(row['Oscar'])))
        cnx.commit() 
    except mysql.connector.Error as err:
        cnx.rollback()  
        print("Error al insertar registro:", err)


mycursor.close()
cnx.close()

 95%|█████████▌| 35303/36981 [01:18<00:02, 710.90it/s]

Error al insertar registro: 1406 (22001): Data too long for column 'Conocido' at row 1


100%|██████████| 36981/36981 [01:21<00:00, 454.77it/s]
